# Poseidon Permutation Truncation Polynomial Model Demonstration

In [1]:
using Oscar
include("Poseidon.jl")

generate_compression_polynomials (generic function with 1 method)

## Poseidon Instance

In [2]:
q = 10007 # d = 3
#q = 10009 # d = 5
field = GF(q)
n = 2
r_f = 2
r_p = 2

poseidon = Poseidon_constructor(field=field, n=n, r_f=r_f, r_p=r_p);

Poseidon parameters
Field: Prime field of characteristic 10007
d: 3
n: 2
r_f: 2
r_p: 2
Matrix:
[7992 4652; 2951 3946]
Constants:
[6162 9238 192 8566 244 314; 1356 1776 730 6278 6668 2678]


In [3]:
x = matrix(map(i -> rand(field), 1:n))

[5003]
[  93]

In [4]:
compr_val = compression(poseidon, x)

1-element Vector{FqFieldElem}:
 3387

In [5]:
polynomials = generate_compression_polynomials(poseidon,
                                               compr_val=compr_val);

sep = repeat("-", 70)
println(sep)

for i in 1:(2 * r_f + r_p)
    for j in 1:n
        println(polynomials[n * (i - 1) + j])
        println(sep)
    end
end

Compression value: FqFieldElem[3387]
substitution: true
----------------------------------------------------------------------
7992*x_5_2^3 + 4652*x_out_1^3 + 10006*x_in_1 + 6162
----------------------------------------------------------------------
2951*x_5_2^3 + 3946*x_out_1^3 + 10006*x_in_2 + 1356
----------------------------------------------------------------------
7992*x_in_1^3 + 4652*x_in_2^3 + 10006*x_1_1 + 9238
----------------------------------------------------------------------
2951*x_in_1^3 + 3946*x_in_2^3 + 10006*x_1_2 + 1776
----------------------------------------------------------------------
7992*x_1_1^3 + 4652*x_1_2 + 10006*x_2_1 + 192
----------------------------------------------------------------------
2951*x_1_1^3 + 3946*x_1_2 + 10006*x_2_2 + 730
----------------------------------------------------------------------
7992*x_2_1^3 + 4652*x_2_2 + 10006*x_3_1 + 8566
----------------------------------------------------------------------
2951*x_2_1^3 + 3946*x_2_2 + 100

## Guess In Last Round

In [6]:
variables = gens(parent(polynomials[1]));
N = length(variables)
guess = map(var -> var - rand(field), variables[N - 2 * n + 1:N - Int64(3 / 2 * n)])

1-element Vector{FqMPolyRingElem}:
 x_4_2 + 6294

In [7]:
gb = groebner_basis(ideal(polynomials))

for poly in gb
    println(poly)
    println(sep)
end

x_2_2 + 9891*x_3_1 + 4770*x_3_2 + 7854
----------------------------------------------------------------------
x_1_2 + 9891*x_2_1 + 2935*x_3_1 + 3018*x_3_2 + 5222
----------------------------------------------------------------------
x_out_1^3 + 9891*x_in_1 + 4770*x_in_2 + 697
----------------------------------------------------------------------
x_5_2^3 + 3389*x_out_1^3 + 2350*x_in_2 + 5633
----------------------------------------------------------------------
x_4_2^3 + 4770*x_5_1 + 6587*x_5_2 + 4335*x_out_1 + 8663
----------------------------------------------------------------------
x_4_1^3 + 3389*x_4_2^3 + 2350*x_5_1 + 7596*x_5_2 + 1660*x_out_1 + 1103
----------------------------------------------------------------------
x_3_2^3 + 9891*x_4_1 + 4770*x_4_2 + 4176
----------------------------------------------------------------------
x_3_1^3 + 3389*x_3_2^3 + 2350*x_4_2 + 1162
----------------------------------------------------------------------
x_2_1^3 + 3389*x_2_2 + 2350*x_3_2 + 7025

In [8]:
gb = groebner_basis(ideal(polynomials) + ideal(guess))

for poly in gb
    println(poly)
    println(sep)
end

x_5_1 + 5286*x_5_2 + 410*x_out_1 + 9837
----------------------------------------------------------------------
x_4_2 + 6294
----------------------------------------------------------------------
x_2_2 + 9891*x_3_1 + 4770*x_3_2 + 7854
----------------------------------------------------------------------
x_1_2 + 9891*x_2_1 + 2935*x_3_1 + 3018*x_3_2 + 5222
----------------------------------------------------------------------
x_out_1^3 + 9891*x_in_1 + 4770*x_in_2 + 697
----------------------------------------------------------------------
x_5_2^3 + 3389*x_out_1^3 + 2350*x_in_2 + 5633
----------------------------------------------------------------------
x_4_1^3 + 2350*x_5_1 + 7596*x_5_2 + 1660*x_out_1 + 4988
----------------------------------------------------------------------
x_3_2^3 + 9891*x_4_1 + 2796
----------------------------------------------------------------------
x_3_1^3 + 3389*x_3_2^3 + 608
----------------------------------------------------------------------
x_2_1^3 + 3389

Compute the vector space dimension.

In [9]:
A, _ = quo(parent(gb[1]), ideal(gb))
D = factor(vector_space_dimension(A))

1 * 3^9

## Guess In Partial Rounds

In [10]:
guess = map(i -> variables[n * (r_f + r_p - i - 1) + 1] - rand(field), 1:Int64(n / 2))

1-element Vector{FqMPolyRingElem}:
 x_2_1 + 1128

In [11]:
gb = groebner_basis_f4(ideal(polynomials) + ideal(guess), nr_thrds=16, info_level=2);


Legend for f4 information
--------------------------------------------------------
deg       current degree of pairs selected in this round
sel       number of pairs selected in this round
pairs     total number of pairs in pair list
mat       matrix dimensions (# rows x # columns)
density   density of the matrix
new data  # new elements for basis in this round
          # zero reductions during linear algebra
time(rd)  time of the current f4 round in seconds given
          for real and cpu time
--------------------------------------------------------

deg     sel   pairs        mat          density            new data         time(rd) in sec (real|cpu)
------------------------------------------------------------------------------------------------------
  3       7       7      15 x 25         15.20%        7 new       0 zero         0.05 | 0.34         
  3       1       1       8 x 12         27.08%        1 new       0 zero         0.02 | 0.16         
  3       1       1       3


--------------- INPUT DATA ---------------
#variables                      13
#equations                      13
#invalid equations               0
field characteristic         10007
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                        16
info level                       2
generate pbm files               0
------------------------------------------

---------------- TIMINGS ----------------
overall(elapsed)        0.33 sec
overall(cpu)            3.82 sec
select                  0.00 sec   0.0%
symbolic prep.          0.00 sec   0.0%
update                  0.00 sec   0.0%
convert                 0.00 sec   0.0%
linear algebra          0.00 sec   0.0%
reduce gb               0.00 sec   0.0%
--------------------------------

In [12]:
for poly in gb
    println(poly)
    println(sep)
end

x_3_1 + 2337*x_3_2 + 1707
----------------------------------------------------------------------
x_2_2 + 5673*x_3_2 + 5726
----------------------------------------------------------------------
x_2_1 + 1128
----------------------------------------------------------------------
x_1_2 + 8725*x_3_2 + 9441
----------------------------------------------------------------------
x_4_1*x_4_2 + 5153*x_4_2^2 + 3460*x_3_2*x_5_1 + 6632*x_4_1*x_5_1 + 6478*x_4_2*x_5_1 + 8065*x_3_2*x_5_2 + 316*x_4_1*x_5_2 + 4771*x_4_2*x_5_2 + 9577*x_3_2*x_out_1 + 4997*x_4_1*x_out_1 + 6019*x_4_2*x_out_1 + 211*x_3_2 + 3677*x_4_1 + 8139*x_4_2 + 6560*x_5_1 + 7593*x_5_2 + 1277*x_out_1 + 2206
----------------------------------------------------------------------
x_3_2*x_4_2 + 5020*x_4_2^2 + 2626*x_3_2*x_5_1 + 4518*x_4_1*x_5_1 + 9758*x_4_2*x_5_1 + 5578*x_3_2*x_5_2 + 8280*x_4_1*x_5_2 + 4002*x_4_2*x_5_2 + 3547*x_3_2*x_out_1 + 9516*x_4_1*x_out_1 + 4546*x_4_2*x_out_1 + 9691*x_3_2 + 3805*x_4_1 + 9687*x_4_2 + 5596*x_5_1 + 642*x_5

Compute the vector space dimension.

In [13]:
A, _ = quo(parent(gb[1]), ideal(gb))
D = factor(vector_space_dimension(A))

1 * 3^7

## Guess Second To Last Round

In [14]:
guess = map(i -> variables[n * (2 * r_f + r_p - 3) + i] - rand(field), 1:Int64(n / 2))

1-element Vector{FqMPolyRingElem}:
 x_3_1 + 7424

In [15]:
gb = groebner_basis_f4(ideal(polynomials) + ideal(guess), nr_thrds=16, info_level=2);


Legend for f4 information
--------------------------------------------------------
deg       current degree of pairs selected in this round
sel       number of pairs selected in this round
pairs     total number of pairs in pair list
mat       matrix dimensions (# rows x # columns)
density   density of the matrix
new data  # new elements for basis in this round
          # zero reductions during linear algebra
time(rd)  time of the current f4 round in seconds given
          for real and cpu time
--------------------------------------------------------

deg     sel   pairs        mat          density            new data         time(rd) in sec (real|cpu)
------------------------------------------------------------------------------------------------------
  3       7       7      15 x 25         15.20%        7 new       0 zero         0.03 | 0.42         
  3       1       1       8 x 13         26.92%        1 new       0 zero         0.01 | 0.14         
  3       1       1       3


--------------- INPUT DATA ---------------
#variables                      13
#equations                      13
#invalid equations               0
field characteristic         10007
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                        16
info level                       2
generate pbm files               0
------------------------------------------

---------------- TIMINGS ----------------
overall(elapsed)        0.11 sec
overall(cpu)            1.59 sec
select                  0.00 sec   0.0%
symbolic prep.          0.00 sec   0.0%
update                  0.00 sec   0.0%
convert                 0.00 sec   0.0%
linear algebra          0.00 sec   0.0%
reduce gb               0.00 sec   0.0%
--------------------------------

In [16]:
for poly in gb
    println(poly)
    println(sep)
end

x_4_1 + 2337*x_4_2 + 875
----------------------------------------------------------------------
x_3_1 + 7424
----------------------------------------------------------------------
x_2_2 + 4770*x_3_2 + 8436
----------------------------------------------------------------------
x_1_2 + 9891*x_2_1 + 3018*x_3_2 + 1021
----------------------------------------------------------------------
x_5_1^2 + 3947*x_4_2*x_5_2 + 6641*x_5_1*x_5_2 + 508*x_5_2^2 + 7692*x_4_2*x_out_1 + 499*x_5_1*x_out_1 + 771*x_5_2*x_out_1 + 4710*x_out_1^2 + 9866*x_4_2 + 1939*x_5_1 + 7034*x_5_2 + 9218*x_out_1 + 604
----------------------------------------------------------------------
x_4_2*x_5_1 + 8324*x_4_2*x_5_2 + 5253*x_4_2*x_out_1 + 8812*x_4_2 + 1275*x_5_1 + 5403*x_5_2 + 5998*x_out_1 + 4056
----------------------------------------------------------------------
x_4_2^2 + 6008*x_4_2 + 4873*x_5_1 + 4481*x_5_2 + 9970*x_out_1 + 1654
----------------------------------------------------------------------
x_out_1^3 + 9891*x_i

Compute the vector space dimension.

In [17]:
A, _ = quo(parent(gb[1]), ideal(gb))
D = factor(vector_space_dimension(A))

1 * 3^8